Comparison
===========

In [ ]:
import sys
import os
sys.path.append(os.path.realpath("../.."))
from scripts.preamble import *
recompute = True

In [ ]:
scene_info = {
    #name: [pretty name, viewpoint]
    'suzanne': ['Suzanne', '25', 9, 0.5*60],
    'bunny': ['Bunny', '25', 9, 0.75*60],
    'bob': ['Bob', '64', 21, 0.5*60],
    'tshirt': ['T-Shirt', '25', 14, 0.2*60],
    'cranium': ['Cranium', '25', 8, 1*60],
    'planck': ['Planck', '25', 11, 0.5*60]
    }
scenes = sorted(scene_info.keys())

In [ ]:
# Load wireframes
wireframes = [[],[],[],[]]
renderings = [[],[],[],[],[]]
losses = [[],[],[]]
res = 100
for i,scene in enumerate(scenes):
    vp = scene_info[scene][2]
    collection = scene_info[scene][1]
    for j, method in enumerate(["base", "res_lap", "res_bilap", "res_smooth"]):
        filename = os.path.join(basename, scene, f"{method}_wireframe.png")
        if recompute or not os.path.exists(filename):
            blender_render(os.path.join(basename, scene, f"{method}.ply"), 
                           os.path.join(basename, scene), 
                           collection,
                           vp,
                           res=100,
                           ours=(j==3),
                           baseline=(j in [1,2]),
                           wireframe=True)
        wireframes[j].append(plt.imread(filename))
        
        if j > 0:
            filename = os.path.join(basename, scene, f"{method}_smooth.png")
            if recompute or not os.path.exists(filename):
                blender_render(os.path.join(basename, scene, f"{method}.ply"), 
                               os.path.join(basename, scene), 
                               collection,
                               vp,
                               res=res,
                               ours=(j==3),
                               baseline=(j in [1,2]))
            renderings[j].append(plt.imread(filename))
            
    for j, method in enumerate(["smooth", "bilap", "lap"]):
            losses[j].append(pd.read_csv(os.path.join(basename, scene, f"loss_{method}.csv"), index_col=0).values)
    
    filename = os.path.join(basename, scene, "ref_smooth.png")
    if recompute or not os.path.exists(filename):
        blender_render(os.path.join(basename, scene, "ref.ply"), 
                       os.path.join(basename, scene), 
                       collection,
                       vp,
                       res=res)
    renderings[-1].append(plt.imread(filename))

In [ ]:
base_size = 3
transpose = True
log = True

if transpose:
    n_cols = len(scenes)
    n_rows = 8
else:
    n_rows = len(scenes)
    n_cols = 8

crop_ratio = 1.3
h,w,_ = renderings[1][0].shape
aspect = w / h
crop_w = int(crop_ratio*h)

widths = np.ones(n_cols) * crop_w / h
heights = np.ones(n_rows)
plot_ratio = 2

if transpose:
    heights[-3:] = crop_ratio/plot_ratio
else:
    widths[-3:] = plot_ratio

crops = np.zeros((len(wireframes[0]), 4), dtype=np.int32)
crops[:,0] = 25000
crops[:,2] = 25000

# Find the closest crop of the rendering to the object
for j in range(len(wireframes)):
    for i, wf in enumerate(wireframes[j]):
        nonzero = np.where(wf[..., 3] > 0)[1]
        crops[i,:2] = (min(crops[i,0], np.min(nonzero)), max(crops[i,1],np.max(nonzero)))
        nonzero = np.where(wf[..., 3] > 0)[0]
        crops[i,2:] = (min(crops[i,2], np.min(nonzero)), max(crops[i,3],np.max(nonzero)))
crop_aspect = (crops[:,1]-crops[:,0])/(crops[:,3]-crops[:,2])

actual_crops=np.zeros_like(crops)
for i, crop in enumerate(crops):
    if crop_aspect[i] < crop_ratio:
        actual_crops[i,2] = crops[i,2]
        actual_crops[i,3] = crops[i,3]
        x0, x1 = crops[i,:2]
        actual_crops[i,0] = (x1-x0)/2*((x1+x0)/(x1-x0) - crop_ratio/crop_aspect[i])
        actual_crops[i,1] = x1+x0-actual_crops[i,0]
    else:
        actual_crops[i,0]=crops[i,0]
        actual_crops[i,1]=crops[i,1]
        y0, y1 = crops[i,2:]
        actual_crops[i,2] = (y1-y0)/2*((y1+y0)/(y1-y0) - crop_aspect[i]/crop_ratio)
        actual_crops[i,3] = y1+y0-actual_crops[i,2]

fig = plt.figure(1, figsize=(widths.sum() * base_size, heights.sum() * base_size), constrained_layout=True)
gs = fig.add_gridspec(n_rows,  n_cols, wspace=0.0,hspace=0.0, height_ratios = heights, width_ratios=widths)

sns.set_style("white")

kw = dict(ha="center", va="center", fontsize=fontsize, color="darkgrey", rotation=45)

for i, scene_name in enumerate(scenes):
    for j, name in enumerate(["Initial State", r"Regularized ($\mathbf{L}$)", r"Regularized ($\mathbf{L}^2$)", r"Ours ($\lambda=19$)", "Target"]):
        if transpose:
            ax = fig.add_subplot(gs[j, i])
        else:
            ax = fig.add_subplot(gs[i, j])

        if j in [1,2,3]:
            wf = wireframes[j][i].copy()
            wf[wf[..., -1] == 0] = 1
            h,w, _ = wf.shape
            I,J = np.meshgrid(np.linspace(0,1,w), np.linspace(0,1,h))
            im = np.ones_like(wf)
            im[I<J] = wf[I<J]
            im[I>J] = renderings[j][i][I>J]
            im = im*im[..., -1][..., None] + (1-im[..., -1][..., None])
            if not np.any(actual_crops[i]<0):
                ax.imshow(im[actual_crops[i,2]:actual_crops[i,3], actual_crops[i,0]:actual_crops[i,1]],)
            else:
                ax.imshow(im[:, (w-crop_w)//2:(w+crop_w)//2],)
        elif j == 4:
            if np.any(actual_crops[i]<0):
                ax.imshow(renderings[j][i][:, (w-crop_w)//2:(w+crop_w)//2],)
            else:
                ax.imshow(renderings[j][i][actual_crops[i,2]:actual_crops[i,3], actual_crops[i,0]:actual_crops[i,1]],)
        else:
            if np.any(actual_crops[i]<0):
                ax.imshow(wireframes[j][i][:, (w-crop_w)//2:(w+crop_w)//2],)
            else:
                ax.imshow(wireframes[j][i][actual_crops[i,2]:actual_crops[i,3], actual_crops[i,0]:actual_crops[i,1]],)
            
        ax.tick_params(axis='both', which='both', labelleft=False, labelbottom=False)
        sns.despine(ax=ax, left=True, bottom=True)
        
        if j == 0:
            if transpose:
                ax.set_title(rf"\textsc{{{scene_info[scene_name][0]}}}", y=1.1, fontsize=fontsize)
            else:
                ax.set_ylabel(scene_info[scene_name][0], x=-1, fontsize=fontsize)
        if i == 0:
            if transpose:
                ax.set_ylabel(name, fontsize=fontsize, x=0)
            else:
                ax.set_title(name, fontsize=fontsize, y=1.1)
        ax.yaxis.set_label_coords(-0.25, 0.5)
        
    with sns.axes_style('darkgrid'):
        for k, loss in enumerate(["MAE", "Laplacian", "Hausdorff"]):
            time = scene_info[scene_name][3]
            if transpose:
                ax = fig.add_subplot(gs[j+k+1, i])
            else:
                ax = fig.add_subplot(gs[i, j+k+1])

            loss_ours = losses[0][i][1::10, k]
            loss_reg = losses[2][i][1::10, k]
            loss_bi = losses[1][i][1::10, k]
            ax.plot(np.linspace(0,time, loss_ours.shape[0]), loss_ours, label=r"Ours ($\lambda=19$)", linewidth=2)
            ax.plot(np.linspace(0,time, loss_reg.shape[0]), loss_reg, label=r"Regularized $\left(\mathbf{L}\right)$", linewidth=2)
            ax.plot(np.linspace(0,time, loss_bi.shape[0]), loss_bi, label=r"Regularized ($\left(\mathbf{L}^2\right)$", linewidth=2)

            if log:
                ax.set_yscale('log')
            else:
                ax.set_ylim(bottom=0)
            if k == 2:
                ax.set_xlabel("Time (s)")
            if k==0 and i==0:
                ax.legend()
            if i == 0:
                if transpose:
                    ax.set_ylabel(loss, fontsize=fontsize)
                else:
                    ax.set_title(loss, y=1.1, fontsize=fontsize)

            ax.yaxis.set_label_coords(-0.25, 0.5)

plt.savefig(os.path.join(basename, "comparison.pdf"), format='pdf', dpi=300, bbox_inches='tight', pad_inches=0.005)